# Time Series Analysis of U.S. Air Traffic Data

## Abstract

The COVID-19 pandemic affected our lives in many ways, one of the biggest being air travel. Restrictions on travel forced many people to stay home, and as a result, the air traffic industry suffered for about a year. Now, in 2024, travel restrictions have been lifted, but how has the pandemic affected air traffic today? This is the research question we will be exploring in this paper. We will be doing a Time Series Analysis on U.S. Air Traffic data from the years 2003 to 2023. We will build a SARIMA model and an LSTM model to forecast the Number of Passengers in upcoming months. We will also run a t-test to determine if there is a difference in the number of p assengers in the period before the pandemic versus the period after the pandemic.

*Keywords: Time Series Analysis, SARIMA, Paired t-test, Long-Short Term Memory (LSTM), Neural Networks.*

## Overview and Motivation

The goal of this project is to explore the trends and seasonality of U.S. air traffic data and to build a forecasting model that can predict the number of passengers in future months. We would like to determine if there is a difference in the number of passengers before the COVID-19 pandemic versus after the pandemic.

## Related Work

This research question was inspired by my own life. Recently, people around me have been taking a lot of trips, seemingly more than before the pandemic. I found this dataset and thought it would be interesting to see if there is actually a statistically significant difference in air travel before the pandemic versus after.

## Data

The dataset we used for this project provides details on the monthly U.S. airline traffic from 2003 to 2023 for all commercial U.S. air carriers. The data comes from the U.S. Department of Transportation’s (DOT) Bureau of Transportation Statistics. The dataset contains 250 observations (each month from the years 2003 to 2023) and has 17 columns. For each month, the dataset contains information on the number of domestic air travel passengers, the number of international air travel passengers, the number of domestic flights, the number of international flights, the number of passengers and the distance flown (Revenue Passenger-miles) for both domestic and international flights, the number of seats and the distance flown (Available Seat-miles) for both domestic and international flights, and the passenger-miles as a proportion of available seat-miles (Load Factor) for both domestic and international flights.

Data cleaning involved transforming data types of multiple columns. We had to remove commas from the values in a few columns to convert them from strings to floats. We also label encoded the years column (0 to 20). For the LSTM model we also normalized the data so that it was in the range 0 to 1.

Source: https://www.kaggle.com/datasets/yyxian/u-s-airline-traffic-data

## Final Research Question

The research question for this project is *How has the COVID-19 pandemic affected the number of passengers on U.S. flights? Can we accurately forecast the number of passengers in future months?* Our independent variable is time and our dependent variable is the number of passengers. Our null hypothesis is that there is a difference in the number of passengers on U.S. flights before the pandemic versus after the pandemic. The alternative hypothesis is that there is no difference in the number of passengers on U.S. flights. We would also like to train a forecasting model to predict the number of passengers on U.S. flights in coming months. We will use the mean absolute percentage error (MAPE) as an evaluation metric.

This research question hasn't changed much over the semester; it was mostly just refined. At first, I thought we might look at domestic versus international flights as well, but it turned out there was much less international data. I also decided to focus on the number of passengers (Pax) variable, as this feature seems to be the most representative of our research question.

## Exploratory Data Analysis

The first step in Exploratory Data Analysis is taking a look at the data and determining what kind of cleaning, if any, needs to be done.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, plot_predict
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
import scipy.stats as stats
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read data in and look at first five rows
df = pd.read_csv('/content/drive/MyDrive/Grad school/Spring 2024/CS 539 - Machine Learning/Data/air_traffic.csv')
df.head()

We will check if there are any missing values. We will also make sure all the feature are the correct data types.

In [ ]:
# Check for missing values
df.isnull().values.any()

In [ ]:
# Check data types
df.dtypes

In [ ]:
# Convert objects to floats
for col in ['Dom_Pax', 'Int_Pax', 'Pax', 'Dom_Flt', 'Int_Flt', 'Flt', 'Dom_RPM', 'Int_RPM', 'RPM', 'Dom_ASM', 'Int_ASM', 'ASM']:
    df[col] = df[col].str.replace(',', '').astype(int)

In [ ]:
# Create column Dates for Month - Year pairs
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
dates = []

for year in range(2003, 2024):
    for month in months:
        dates.append(month + ' ' + str(year))

df['Date'] = dates[:249]

In [ ]:
# Encode years
le = LabelEncoder()
df['Year'] = le.fit_transform(df['Year'])

Now that the data is clean, let's look at some summary statistics.

In [ ]:
# Look at summary statistics
df.describe()

Lets plot the distributions of each feature.

In [ ]:
# Plot histograms for the non-time features
%matplotlib inline

pos = []
for n in range(5):
    for m in range(3):
         pos.append((n,m))

fig, axs = plt.subplots(5, 3, figsize=(14, 20))
features = ['Dom_Pax', 'Int_Pax', 'Pax', 'Dom_Flt', 'Int_Flt', 'Flt', 'Dom_RPM', 'Int_RPM', 'RPM', 'Dom_ASM', 'Int_ASM', 'ASM', 'Dom_LF', 'Int_LF', 'LF']

for i in range(15):
    col = features[i]
    axs[pos[i]].hist(df[col])
    axs[pos[i]].set_title(col)

All the histograms seems skewed a bit to the left. This is likely due to the year of 2020 during the COVID-19 pandemic when there was much less air traffic than normal. These are likely outliers, and we can deal with them before we create a forecasting model.

In [ ]:
# Plot variables over time
fig, axs = plt.subplots(5, 1, figsize=(8, 30))
plt.setp(axs, xticks=(range(0, len(df['Date']), 60)), xticklabels=range(2003, 2024, 5))
features = ['Pax', 'Flt', 'RPM', 'ASM', 'LF']
labels = ['Number of Passengers', 'Number of Flights', 'Revenue Passenger-Miles', 'Available Seat-Miles', 'Load Factor']

for i in range(5):
  dom = 'Dom_' + features[i]
  intl = 'Int_' + features[i]
  total = features[i]
  axs[i].plot(df['Date'], df[dom], label='Domestic')
  axs[i].plot(df['Date'], df[intl], label='International')
  axs[i].plot(df['Date'], df[total], label='Total')
  axs[i].set_title(labels[i])
  axs[i].legend()

There are more Domestic flights than International. We can see this in all the variables except Load Factor. We can also again see the effects of the COVID-19 pandemic. Ignoring that time period, we can see that overall the number of flights is going down while all the other variables are going up, including number of passengers.

In [ ]:
# Plot variables over time for each year
fig, axs = plt.subplots(5, 1, figsize=(12, 50))

for n in range(5):
    for i in range(21):
        axs[n].plot(df['Month'].loc[df['Year'] == i], df[features[n]].loc[df['Year'] == i], label=2003+i)
        axs[n].set_title(labels[n])

    axs[n].set_xticks(range(1, 13))
    axs[n].set_xticklabels(months, rotation=90)
    axs[n].legend(bbox_to_anchor=(1.05, 1))

In these graphs you can see the overall trends in flights throughout the year. You can see that air travel starts to pick up in the spring and peaks in the summer, dropping off again around September, and finally picking up again during the holidays.

These graphs are a bit difficult to read; let's see if we can change the colors to a gradient.

In [ ]:
# Set colormap
colors = plt.cm.Blues(np.linspace(0, 1, 21))

for n in range(5):
    plt.figure(figsize=(10, 8))
    plt.xticks(range(1, 13), months, rotation='vertical')
    plt.title(labels[n])

    for i in range(21):
        plt.plot(df['Month'].loc[df['Year'] == i], df[features[n]].loc[df['Year'] == i], label=2003+i, color=colors[i])
    plt.legend(bbox_to_anchor=(1.05, 1))
    plt.show()

Now we can see the trends over the years. again we see the number of flights goes down over the years while all the other variables increase over the years. We can see where air travel drops off during the COVID-19 pandemic in March/April of 2020 and where it slowly picks up again the following year. We can also see that number of passengers is at a peak this past year (2023), as is Revenue Passenger-Miles and Available Seat-Miles.

In [ ]:
# Plot correlation matrix to see which variables are correlated with each other
# For this we are only considering the overall variables (i.e., not the Domestic and International variables)
matrix = df[['Month', 'Year', 'Pax', 'Flt', 'RPM', 'ASM', 'LF']].corr()

plt.imshow(matrix, cmap='Blues')
plt.colorbar()

variables = []
for i in matrix.columns:
    variables.append(i)

plt.xticks(range(len(matrix)), variables, rotation=45, ha='right')
plt.yticks(range(len(matrix)), variables)

plt.show()

As we could already kind of see in the previous trends, the number of passengers is correlated to the Revenue Passenger-Miles, the Available Seat-Miles, and the Load Factor. This makes sense since these variables are all dependent on the number of passengers.

In [ ]:
years = df['Year'].unique()

# Draw Plot
fig, axes = plt.subplots(1, 2, figsize=(20,7), dpi= 80)
sns.boxplot(x='Year', y='Pax', data=df, ax=axes[0])
sns.boxplot(x='Month', y='Pax', data=df.loc[~df['Year'].isin([0, 18]), :])

# Set Title
axes[0].set_title('Number of Passengers (Trend)', fontsize=18)
axes[0].set_xticklabels(range(2003, 2024))
axes[0].tick_params(axis='x', labelrotation=90)
axes[1].set_title('Number of Passengers (Seasonality)', fontsize=18)
plt.show()

Finally, from the boxplots above, we can see that there is a positive trend (other than the years 2020 and 2021) and there is yearly seasonality.

## Final Models

We want to see if there is a difference in the number of passengers before the COVID-19 pandemic versus after the pandemic. To do this, we will run a paired t-test on the number of passengers in the months before the pandemic and the months after. For this test, we will say that the period of restricted air travel lasted from 2020 through 2021 (it likely lifted sooner, but we will use a wider interval to be safe). We will look at the year before this period (2019) and the year after (2022). The null hypothesis is that there is no difference in the number of passengers before the pandemic and after the pandemic. The alternative hypothesis is that there is a difference.

In [ ]:
# Run t-test to check if the number of passengers in the years before the COVID-19 pandemic is statistically
# different from the number of passengers in the years after the pandemic.
pre_covid = df[df['Year'] == 16]
post_covid = df[df['Year'] == 19]
pre_covid = pre_covid[:len(post_covid)]

stats.ttest_rel(pre_covid['Pax'], post_covid['Pax'])

The p-value is less than $\alpha=0.05$, so we have sufficient evidence to reject the null hypothesis that the number of passengers is the same in these two periods. Therefore, we can conclude that there is a difference in the number of passengers before the pandemic versus after the pandemic. However, we don't know what this difference is.

Let's see if we can create a model to forecast the number of passengers in coming months. Before building our model, we have to split the data into train and test sets.

In [ ]:
# Split into train and test
test = df.loc[df['Year'] >= 19]
train = df.loc[df['Year'] < 19]

We can decompose the data into its three componenets: trend, seasonality, and residual.

In [ ]:
# Additive Decomposition
additive_decomposition = seasonal_decompose(train['Pax'], model='additive', period=12)

# Multiplicative Decomposition
multiplicative_decomposition = seasonal_decompose(train['Pax'], model='multiplicative', period=12)

# Plot
plt.rcParams.update({'figure.figsize': (16,12)})
additive_decomposition.plot().suptitle('Additive Decomposition', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

multiplicative_decomposition.plot().suptitle('Multiplicative Decomposition', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

plt.show()

While it is hard to tell, the additive decomposition looks a bit more random, so we will use this decomposition to test for outliers in the residuals.

In [ ]:
plt.rc('figure',figsize=(12,6))
plt.rc('font',size=15)
fig, ax = plt.subplots()
x = additive_decomposition.resid.index
y = additive_decomposition.resid.values
ax.plot(x, y)
ax.hlines(y=1e7, xmin=0, xmax=249, color='r')
ax.hlines(y=-1e7, xmin=0, xmax=249, color='r')
ax.set_xticks((range(0, len(df['Date']), 24)))
ax.set_xticklabels(range(2003, 2024, 2))
ax.set_title('Residual Outliers')
plt.show()

In order to deal with the COVID-19 outliers, we are going to try replacing the outliers with the mean number of passengers for that month (not including the outliers in the mean calculation).

In [ ]:
# Convert outliers to the mean for each month (not including the outliers)
outliers = list(x[abs(y) > 1e7])
do_train = train.drop(labels=outliers)

for n in outliers:
    month = train.iloc[n]['Month']
    train.loc[n, train.columns.get_loc('Pax')] = do_train[do_train['Month'] == month]['Pax'].mean()

Now, we need to check for stationarity. We do this using the Augmented Dicky-Fuller Test.

In [ ]:
# Run Augmented Dicky-Fuller Test
results = adfuller(train['Pax'])
print(f'ADF Statistic: {results[0]}')
print(f'p-value: {results[1]}')
print('Critical Values:')
for key, value in results[4].items():
    print(f'{key}: {value}')

Our p-value for this test is less than $\alpha=0.05$, but the ADF statistic isn't significantly smaller than the critical values. Let's see what happens when we difference the data.

In [ ]:
# Transform time series by differencing
pax_transformed = train['Pax'].diff().dropna()
results = adfuller(pax_transformed)
print(f'ADF Statistic: {results[0]}')
print(f'p-value: {results[1]}')
print('Critical Values:')
for key, value in results[4].items():
    print(f'{key}: {value}')

This seems much better. Now let's plot the Autocorrelation and Partial Autocorrelation functions.

In [ ]:
# Plot Autocorrelation
plot_acf(train['Pax'], lags=50)
plt.show()

In [ ]:
# Plot Partial Autocorrelation
plot_pacf(train['Pax'], lags=50)
plt.show()

We can see that there is definitely yearly seasonality in the data. We can use these plots to determine $p$ and $q$, or we can try a grid search. This way tests out all the combinations of different hyperparameters to find the best combination based on the AIC. This will likely give us better results, so let's do this.

In [ ]:
# Initialize parameters
p_params = range(0, 3)
d_params = [0, 1]
q_params = range(0, 3)
P_params = range(0, 3)
D_params = [0, 1]
Q_params = range(0, 3)

params = {'order': [], 'seasonal_order': []}
for p in p_params:
    for d in d_params:
        for q in q_params:
            order = (p, d, q)
            params['order'].append(order)

for P in P_params:
    for D in D_params:
        for Q in Q_params:
            seasonal_order = (P, D, Q, 12)
            params['seasonal_order'].append(seasonal_order)

best_aic = 100000
best_params = None
for order in params['order']:
    for seasonal_order in params['seasonal_order']:
        model = SARIMAX(train['Pax'], order=order, seasonal_order=seasonal_order)
        model_fit = model.fit()
        aic = model_fit.aic
        preds = model_fit.predict()
        if aic < best_aic:
            best_aic = aic
            best_params = [order, seasonal_order]

print(f'Best AIC: {best_aic}')
print(f'Best parameters: {best_params}')

From the grid search, we found that the best combination is $order=(2, 1, 0)$ and $seasonal\:order=(0, 1, 1, 12)$. Let's try building our SARIMA model with these parameters.

In [ ]:
# Build SARIMA model
model = SARIMAX(train['Pax'], order=(2, 1, 0), seasonal_order=(0, 1, 1, 12))
model_fit = model.fit()
preds = model_fit.predict()
model_fit.summary()

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
# Plot Actual vs Fitted
fig, ax = plt.subplots(figsize=(10, 7))
ax = train['Pax'].plot(ax=ax)
ax.plot(test['Pax'])
ax.set_xticks((range(0, len(df['Date']), 48)))
ax.set_xticklabels(range(2003, 2024, 4))
plot_predict(model_fit, 175, 249, ax=ax)
plt.show()

In [ ]:
print(f"Train MAPE: {mean_absolute_percentage_error(train['Pax'], preds)}")
preds = model_fit.predict(229, 249)
print(f"Test MAPE: {mean_absolute_percentage_error(test['Pax'], preds)}")

This model seems to do pretty well on the training data. The mean absolute percentage error is small, indicating the model is performing very well. The residuals are pretty random, and the forecast is fairly close to the actual data, as seen in the plot above.

The model also does well on the test data, but is not as accurate as we would like. There is also a large confidence interval. Let's try one more model to see if we can do better. For the second model, we are going to try an LSTM (Long-Short Term Memory) neural network. First, we have to do a little preprocessing to the data. LSTMs are very sensitive to the scale of input data, so we normalize the data so that it is in the range $[0, 1]$.

In [ ]:
tf.random.set_seed(12)

dataframe = pd.concat([train['Pax'], test['Pax']])
dataset = dataframe.values
dataset = dataset.astype('float32')
dataset = dataset.reshape(-1, 1)

In [ ]:
# Normalize the dataset
scaler = MinMaxScaler()
dataset = scaler.fit_transform(dataset)

In [ ]:
# Split into train and test
train, test = dataset[0:228,:], dataset[228:len(dataset),:]

We create a function that turns the data into two arrays, the first array being the number of passengers of the current month, and the second array is the number of passengers of the next month. This is how we will feed the data into the LSTM model.

In [ ]:
# Convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []

    for i in range(len(dataset)-look_back-1):
      a = dataset[i:(i+look_back), 0]
      dataX.append(a)
      dataY.append(dataset[i + look_back, 0])

    return np.array(dataX), np.array(dataY)

In [ ]:
# Reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

Now that the data is formatted correctly, we can create and fit our LSTM model.

In [ ]:
# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

Let's see how it does on the train and test sets.

In [ ]:
# Make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# Calculate mean absolute percentage error
print(f"Train MAPE: {mean_absolute_percentage_error(trainY[0], trainPredict[:,0])}")
print(f"Test MAPE: {mean_absolute_percentage_error(testY[0], testPredict[:,0])}")

In [ ]:
# Shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# Shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

# Plot baseline and predictions
fig, ax = plt.subplots()
a = ax.plot(scaler.inverse_transform(dataset))
b = ax.plot(trainPredictPlot)
c = ax.plot(testPredictPlot)
ax.set_xticks((range(0, len(df['Date']), 48)))
ax.set_xticklabels(range(2003, 2024, 4))
ax.legend([a, b, c], labels=['Actual', 'Train Forecast', 'Test Forecast'])
plt.show()

In [ ]:
def plot_loss(history):
    plt.figure(figsize=(10, 7))
    plt.plot(history.history['loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()

plot_loss(history)

This model seems to do pretty well on our data. The mean absolute percentage error is about 13% for the training data, which is good, and about 6.7% for the test data, which is excellent. The difference in these values (normally the train would be smaller) might be due to the size of my test set. It might have been better to test on a larger sample, but I didn't want the COVID-19 data to be in the test set. We also plotted the loss to make sure the model is converging, which it is.

## Final Analysis

Both of these forecasting models do a pretty good job at predicting the Number of Passengers. The LSTM model does a bit better on the test dataset. One interesting thing we noticed is that the model's prediction on the test set is lower than the actual values for both models. This may indicate that there is, in fact, an increase in air travel that even the model might not realize due to the fact that it was trained on past data.

Another interesting thing we found in our EDA is that the number of flights decreases over the years while the number of passengers increases. This might mean that airplanes are able to hold more passengers, or that flights are more often fully booked. We also noticed that there are much less international flights than domestic flights. Finally, we saw the yearly trends of the data, noticing how the number of passengers increases in the spring and summer as people begin traveling, and then drops off around September when kids are starting to go back to school. There is another increase around Christmas, as you'd expect.

Our paired t-test showed that there is a difference in the Number of Passengers in the period before the COVID-19 pandemic versus the period after the pandemic. From the EDA and the forecasting predictions, it seems that this difference is an increase.


In [ ]:
# Colab2PDF v1.0.4 by Drengskapur (github.com/drengskapur/colab2pdf) (License: GPL-3.0-or-later)
# @title {display-mode:"form"}
# @markdown ⬇️ Download PDF
def colab2pdf():
    ENABLE=True # @param {type:"boolean"}
    if ENABLE:
        !apt-get install librsvg2-bin
        import os, datetime, json, locale, pathlib, urllib, requests, werkzeug, nbformat, google, yaml, warnings
        locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
        NAME = pathlib.Path(werkzeug.utils.secure_filename(urllib.parse.unquote(requests.get(f"http://{os.environ['COLAB_JUPYTER_IP']}:{os.environ['KMP_TARGET_PORT']}/api/sessions").json()[0]["name"])))
        TEMP = pathlib.Path("/content/pdfs") / f"{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_{NAME.stem}"; TEMP.mkdir(parents=True, exist_ok=True)
        NB = [cell for cell in nbformat.reads(json.dumps(google.colab._message.blocking_request("get_ipynb", timeout_sec=600)["ipynb"]), as_version=4).cells if "--Colab2PDF" not in cell.source]
        warnings.filterwarnings('ignore', category=nbformat.validator.MissingIDFieldWarning)
        with (TEMP / f"{NAME.stem}.ipynb").open("w", encoding="utf-8") as nb_copy: nbformat.write(nbformat.v4.new_notebook(cells=NB or [nbformat.v4.new_code_cell("#")]), nb_copy)
        if not pathlib.Path("/usr/local/bin/quarto").exists():
            !wget -q "https://quarto.org/download/latest/quarto-linux-amd64.deb" -P {TEMP} && dpkg -i {TEMP}/quarto-linux-amd64.deb > /dev/null && quarto install tinytex --update-path --quiet
        with (TEMP / "config.yml").open("w", encoding="utf-8") as file: yaml.dump({'include-in-header': [{"text": r"\usepackage{fvextra}\DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaksymbolleft={},showspaces=false,showtabs=false,breaklines,breakanywhere,commandchars=\\\{\}}"}],'include-before-body': [{"text": r"\DefineVerbatimEnvironment{verbatim}{Verbatim}{breaksymbolleft={},showspaces=false,showtabs=false,breaklines}"}]}, file)
        !quarto render {TEMP}/{NAME.stem}.ipynb --metadata-file={TEMP}/config.yml --to pdf -M latex-auto-install -M margin-top=1in -M margin-bottom=1in -M margin-left=1in -M margin-right=1in --quiet
        google.colab.files.download(str(TEMP / f"{NAME.stem}.pdf"))
colab2pdf()